In [ ]:
import sys, warnings
sys.path.append('..')
warnings.filterwarnings('ignore')

import numpy as np
import cv2
import scipy.signal as signal
import matplotlib.pyplot as plt
from tqdm import trange

In [ ]:
def rgb2yiq(x):
    rgb = (x[..., ::-1]).astype(np.float32)
    y = rgb @ np.array([[0.30], [0.59], [0.11]])
    rby = rgb[:, :, (0,2)] - y
    i = np.sum(rby * np.array([[[0.74, -0.27]]]), axis=-1)
    q = np.sum(rby * np.array([[[0.48, 0.41]]]), axis=-1)
    return np.dstack((y.squeeze(), i, q))

In [ ]:
def yiq2rgb(yiq):
    r = yiq @ np.array([1.0, 0.9468822170900693, 0.6235565819861433])
    g = yiq @ np.array([1.0, -0.27478764629897834, -0.6356910791873801])
    b = yiq @ np.array([1.0, -1.1085450346420322, 1.7090069284064666])
    rgb = np.clip(np.dstack((r, g, b)), 0, 1)
    return rgb

inv_colorspace = lambda x: cv2.normalize(yiq2rgb(x), None, 0, 255.0, cv2.NORM_MINMAX, cv2.CV_8UC3)

In [ ]:
DATA = 'data/face.mp4'

In [ ]:
ALPHA = 50.0 # Magnification factor
LEVEL = 4 # Gaussian Pyramid

# Temporal filter parameters
f_lo = 50/60 # 0.83
f_hi = 60/60 # 1.0

In [ ]:
cap = cv2.VideoCapture(DATA)

w, h = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

print(f"Width: {w} | Height: {h}")
print(f"Detected Video Sampling Rate (FPS): {fps}")

frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frames.append(rgb2yiq(frame / 255.0))
    else:
        break

In [ ]:
NUM_FRAMES = len(frames)

In [ ]:
bandpass = signal.firwin(numtaps=len(frames), cutoff=(f_lo, f_hi), fs=fps, pass_zero=False)
transfer_function = np.fft.fft(np.fft.ifftshift(bandpass))

In [ ]:
norm_freqs, response = signal.freqz(bandpass)
freqs = norm_freqs / np.pi * fps / 2

In [ ]:
_, ax = plt.subplots(1, 4, figsize=(20, 5))
ax[0].plot(np.abs(transfer_function))
ax[0].set_title("Transfer Function")
ax[0].set_xlim([0, len(frames)])
ax[0].grid(alpha=0.25)

ax[1].plot(bandpass)
ax[1].set_title("Impulse Response")
ax[1].set_xlim([0, len(frames)])
ax[1].grid(alpha=0.25)

ax[2].plot(freqs, 20 * np.log10(np.abs(response)))
ax[2].plot([f_lo, f_lo], [-100, 0], color='r', lw=0.5)
ax[2].plot([f_hi, f_hi], [-100, 0], color='r', lw=0.5)
ax[2].set_title("Frequency Response")
ax[2].set_ylabel("Amplitude")
ax[2].set_xlim([0, 15]), ax[0].grid(alpha=0.25)
ax[2].grid(alpha=0.25)

ax[3].plot(freqs, np.angle(response))
ax[3].set_title("Phase Response")
ax[3].set_xlabel("Freqeuncy (Hz)")
ax[3].set_ylabel("Angle (radians)")
ax[3].set_xlim([0, 15]), ax[1].grid(alpha=0.25)
ax[3].grid(alpha=0.25)

plt.tight_layout()

In [ ]:
def gaussian_pyramid(image, level):
    r, c, ch = image.shape
    scale = 2**level
    pyramid = np.zeros((ch, r//scale, c//scale))

    for i in range(0, level):
        image = cv2.pyrDown(image, dstsize=(c//2, r//2))
        r, c, _ = image.shape

        if i==(level-1):
            for c in range(ch):
                pyramid[c,:,:] = image[:,:,c]

    return pyramid

In [ ]:
r, c, ch = frames[0].shape
scale = 2**LEVEL
pyramid_stack = np.zeros((len(frames), ch, r//scale, c//scale))

In [ ]:
for i, frame in enumerate(frames):
    pyramid = gaussian_pyramid(frame, LEVEL)
    pyramid_stack[i,:,:,:] = pyramid

In [ ]:
plt.figure(figsize=(15, 5))
plt.imshow(pyramid_stack[0,:,:,:].transpose(1,0,2).reshape((pyramid.shape[1],-1)), cmap='gray')
plt.axis(False), plt.tight_layout()

In [ ]:
pyr_stack_fft = np.fft.fft(pyramid_stack, axis=0).astype(np.complex64)
_filtered_pyramid = pyr_stack_fft * transfer_function[:, None, None, None].astype(np.complex64)
filtered_pyramid = np.fft.ifft(_filtered_pyramid, axis=0).real

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 5), sharey=True)

ax[0].plot(np.abs(pyr_stack_fft[2:-2, 0, 20, 12]))
ax[0].set_title("Unfiltered Signal at (20, 12)")
ax[0].set_xlim([0, len(frames)])
ax[0].grid(alpha=0.25)

ax[1].plot(np.abs(_filtered_pyramid[2:-2, 0, 20, 12]))
ax[1].set_title("Filtered Signal at (20, 12)")
ax[1].set_xlim([0, len(frames)])
ax[1].grid(alpha=0.25)

plt.tight_layout()

In [ ]:
_, ax = plt.subplots(1, 2,figsize=(8, 5))
ax[0].imshow(pyramid_stack[50,0,:,:], cmap='gray')
ax[0].set_title("Unfiltered Luma Channel")
ax[0].axis(False)

ax[1].imshow(filtered_pyramid[50,0,:,:], cmap='gray')
ax[1].set_title("Filtered Luma Channel")
ax[1].axis(False)

plt.tight_layout()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(pyramid_stack[:, 0, 12, 20] - pyramid_stack[:, 0, 12, 20].mean())
plt.plot(filtered_pyramid[:, 0, 12, 20])
plt.xlim([0, len(frames)])
plt.grid(alpha=0.25), plt.tight_layout()

In [ ]:
magnified_pyramid = filtered_pyramid * ALPHA

In [ ]:
magnified = []
magnified_only = []

for i in (t := trange(len(frames))):
    y_chan = frames[i][:, :, 0]
    i_chan = frames[i][:, :, 1] 
    q_chan = frames[i][:, :, 2] 
    
    fy_chan = cv2.resize(magnified_pyramid[i, 0, :, :], (c, r))
    fi_chan = cv2.resize(magnified_pyramid[i, 1, :, :], (c, r))
    fq_chan = cv2.resize(magnified_pyramid[i, 2, :, :], (c, r))

    mag = np.dstack((y_chan + fy_chan, i_chan + fi_chan, q_chan + fq_chan,))
    mag = inv_colorspace(mag)

    magnified.append(mag)
    magnified_only.append(np.dstack((fy_chan, fi_chan, fq_chan)))

In [ ]:
x_r, x_g, x_b = [], [], []

red, green, blue = [], [], []
for i in (t := trange(len(frames))):
    frame = inv_colorspace(frames[i])
    x_r.append(frame[0, :, :].sum())
    x_b.append(frame[1, :, :].sum())
    x_g.append(frame[2, :, :].sum())

    red.append(magnified[i][0, :, :].sum())
    blue.append(magnified[i][1, :, :].sum())
    green.append(magnified[i][2, :, :].sum())

In [ ]:
times = np.arange(0, len(frames))/fps
_, ax = plt.subplots(1, 2, figsize=(15, 5), sharey=True)

ax[0].plot(times, x_r, color='red')
ax[0].plot(times, x_b, color='blue')
ax[0].plot(times, x_g, color='green')
ax[0].set_title("Original", size=18)
ax[0].set_xlabel("Time", size=16)
ax[0].set_ylabel("Intensity", size=16)
ax[0].set_xlim([0, len(frames)/fps])
ax[0].grid(alpha=0.25)

ax[1].plot(times, red, color='red')
ax[1].plot(times, blue, color='blue')
ax[1].plot(times, green, color='green')
ax[1].set_title("Filtered", size=18)
ax[1].set_xlabel("Time", size=16)
ax[1].set_xlim([0, len(frames)/fps])
ax[1].grid(alpha=0.25)

plt.tight_layout()

In [ ]:
freqs = np.fft.rfftfreq(len(frames)) * fps
rates = np.abs(np.fft.rfft(red))/len(frames)

In [ ]:
plt.plot(freqs[1:], rates[1:])
plt.title("DFT of Red channel Intensities")
plt.xlabel("Freuqency")
plt.ylabel("Amplitude")
plt.xlim([0, len(freqs[1:])/10])
plt.grid(alpha=0.25), plt.tight_layout()

In [ ]:
peak_idx, _ = signal.find_peaks(rates, height=1000)

In [ ]:
bpm = freqs[peak_idx].squeeze(0) * 60
print(f"BPM: {bpm}")

In [ ]:
stacked_frames = []
middle = np.zeros((r, 3, 3)).astype(np.uint8)

for vid_idx in range(len(frames)):
    og_frame = cv2.normalize(yiq2rgb(frames[vid_idx]), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC3)
    frame = np.hstack((cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR), middle, cv2.cvtColor(magnified[vid_idx], cv2.COLOR_RGB2BGR)))
    stacked_frames.append(frame)

In [ ]:
plt.imshow(cv2.cvtColor(stacked_frames[10], cv2.COLOR_BGR2RGB))
plt.axis(False), plt.tight_layout()

In [ ]:
_h, _w, _ = stacked_frames[-1].shape
out = cv2.VideoWriter(f"bpm_alpha{int(ALPHA)}.mp4", cv2.VideoWriter_fourcc(*'MP4V'), int(fps), (_w, _h))
 
for frame in stacked_frames:
    out.write(frame)
out.release()

In [ ]:
_h, _w, _ = magnified_only[-1].shape
out = cv2.VideoWriter(f"bpm_signal.mp4", cv2.VideoWriter_fourcc(*'MP4V'), int(fps), (_w, _h))

sums = []
for frame in magnified_only:
    sums.append(frame.sum(axis=1).sum(axis=0))
    
    frame = cv2.cvtColor(
        cv2.normalize(frame*20, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1),
        cv2.COLOR_RGB2BGR)
    out.write(frame)
out.release()

In [ ]:
stacked = np.array([cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) for frame in stacked_frames])

In [ ]:
idx = 220 

_, ax = plt.subplots(1, 2, figsize=(10,10))
ax[0].imshow(stacked[:, :, idx, :].transpose(1, 0, 2))
ax[0].set_title("Original Image")

ax[1].imshow(stacked[:, :, (idx + w + 3), :].transpose(1, 0, 2))
ax[1].set_title("Color Magnified")

plt.tight_layout()